In [1]:
import tensorflow as tf
print(tf.__version__)

1.15.0


In [10]:
!pip install keras

  Using cached Keras-2.3.1-py2.py3-none-any.whl (377 kB)
  Using cached PyYAML-5.3.1.tar.gz (269 kB)
  Using cached Keras_Applications-1.0.8-py3-none-any.whl (50 kB)
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp38-cp38-linux_x86_64.whl size=44617 sha256=1e78c6268529cef11e18c752d2c34068b51c9fe46b5e2878b4de7614d880017e
  Stored in directory: /home/team/.cache/pip/wheels/13/90/db/290ab3a34f2ef0b5a0f89235dc2d40fea83e77de84ed2dc05c
Successfully built pyyaml


In [12]:
!pip install tensorflow

     |▏                               | 3.2 MB 173 kB/s eta 0:49:17^C

ERROR: Operation cancelled by user


In [3]:
import tensorflow as tf
import keras
from tensorflow.python.client import device_lib
from keras import backend

Using TensorFlow backend.


In [4]:
# 케라스와 텐서플로우 버전 확인
print(keras.__version__)
print(tf.__version__)

2.3.1
2.2.0


In [5]:
# gpu를 인식하는지 확인합니다. 모두 True가 나오면 됩니다.
print(tf.test.is_built_with_cuda())
print(tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None))
#print(tf.config.list_pysical_devices('GPU'))

True
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True


In [6]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [8]:
!conda install matplotlib

Solving environment: done

## Package Plan ##

  environment location: /home/team/anaconda3/envs/young

  added / updated specs:
    - matplotlib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cycler-0.10.0              |           py38_0          14 KB
    kiwisolver-1.2.0           |   py38hfd86e86_0          84 KB
    matplotlib-3.1.3           |           py38_0          21 KB
    matplotlib-base-3.1.3      |   py38hef1b27d_0         5.0 MB
    pyparsing-2.4.7            |             py_0          65 KB
    ------------------------------------------------------------
                                           Total:         5.2 MB

The following NEW packages will be INSTALLED:

  cycler             pkgs/main/linux-64::cycler-0.10.0-py38_0
  kiwisolver         pkgs/main/linux-64::kiwisolver-1.2.0-py38hfd86e86_0
  matplotlib         pkgs/main/linux-64::matplotlib-3.1.3-py38_0
  mat

In [11]:
from keras import models, layers
from keras import Input
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers, initializers, regularizers, metrics
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import BatchNormalization, Conv2D, Activation, Dense, GlobalAveragePooling2D, MaxPooling2D, ZeroPadding2D, Add
 
import os
#import matplotlib.pyplot as plt
import numpy as np
import math
 

In [13]:
'''
train_datagen = ImageDataGenerator(featurewise_center=False,
                 samplewise_center=False,
                 featurewise_std_normalization=False,
                 samplewise_std_normalization=False,
                 zca_whitening=False,
                 rotation_range=5,
                 width_shift_range=0.05,
                 height_shift_range=0.05,
                 shear_range=0.2,
                 zoom_range=0.2,
                 channel_shift_range=0.,
                 fill_mode='nearest',
                 cval=0.,
                 horizontal_flip=True,
                 vertical_flip=False,
                 rescale=1/255,
                validation_split=0.33
                                  ) #rescale to [0-1], add zoom range of 0.2x and horizontal flip

'''
train_datagen = ImageDataGenerator(rescale=1/255,validation_split=0.33)
PATH="/home/team/바탕화면/data_final"
train_generator = train_datagen.flow_from_directory(
        PATH,
        shuffle=True,
       seed=13,
    class_mode = 'categorical',
        target_size=(224,224),
        batch_size=64,
subset="training")



val_generator = train_datagen.flow_from_directory(
        PATH,
        shuffle=True,
        seed=13,
        target_size=(224,224),
        batch_size=64,
        class_mode = 'categorical',
        subset="validation")

 
 
# number of classes
K = 4
 
 

Found 13348 images belonging to 4 classes.
Found 6572 images belonging to 4 classes.


In [14]:
input_tensor = Input(shape=(224, 224, 3), dtype='float32', name='input')

In [15]:
def conv1_layer(x):    
    x = ZeroPadding2D(padding=(3, 3))(x)
    x = Conv2D(64, (7, 7), strides=(2, 2))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = ZeroPadding2D(padding=(1,1))(x)
 
    return x   
 

In [6]:
def conv2_layer(x):         
    x = MaxPooling2D((3, 3), 2)(x)     
 
    shortcut = x
 
    for i in range(3):
        if (i == 0):
            x = Conv2D(64, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(64, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(shortcut)            
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)
 
            x = Add()([x, shortcut])
            x = Activation('relu')(x)
            
            shortcut = x
 
        else:
            x = Conv2D(64, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(64, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)            
 
            x = Add()([x, shortcut])   
            x = Activation('relu')(x)  
 
            shortcut = x        
    
    return x
 

In [7]:
def conv3_layer(x):        
    shortcut = x    
    
    for i in range(4):     
        if(i == 0):            
            x = Conv2D(128, (1, 1), strides=(2, 2), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)        
            
            x = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)  
 
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(512, (1, 1), strides=(2, 2), padding='valid')(shortcut)
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)            
 
            x = Add()([x, shortcut])    
            x = Activation('relu')(x)    
 
            shortcut = x              
        
        else:
            x = Conv2D(128, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)            
 
            x = Add()([x, shortcut])     
            x = Activation('relu')(x)
 
            shortcut = x      
            
    return x

In [8]:
def conv4_layer(x):
    shortcut = x        
  
    for i in range(6):     
        if(i == 0):            
            x = Conv2D(256, (1, 1), strides=(2, 2), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)        
            
            x = Conv2D(256, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)  
 
            x = Conv2D(1024, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(1024, (1, 1), strides=(2, 2), padding='valid')(shortcut)
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)
 
            x = Add()([x, shortcut]) 
            x = Activation('relu')(x)
 
            shortcut = x               
        
        else:
            x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(256, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(1024, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)            
 
            x = Add()([x, shortcut])    
            x = Activation('relu')(x)
 
            shortcut = x      
 
    return x

In [9]:
def conv5_layer(x):
    shortcut = x    
  
    for i in range(3):     
        if(i == 0):            
            x = Conv2D(512, (1, 1), strides=(2, 2), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)        
            
            x = Conv2D(512, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)  
 
            x = Conv2D(2048, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(2048, (1, 1), strides=(2, 2), padding='valid')(shortcut)
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)            
 
            x = Add()([x, shortcut])  
            x = Activation('relu')(x)      
 
            shortcut = x               
        
        else:
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(512, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(2048, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)           
            
            x = Add()([x, shortcut]) 
            x = Activation('relu')(x)       
 
            shortcut = x                  
 
    return x
 

In [10]:

x = conv1_layer(input_tensor)
x = conv2_layer(x)
x = conv3_layer(x)
x = conv4_layer(x)
x = conv5_layer(x)

In [11]:
x = GlobalAveragePooling2D()(x)
output_tensor = Dense(K, activation='softmax')(x)

In [12]:
resnet50 = Model(input_tensor, output_tensor)
resnet50.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 230, 230, 3)  0           input[0][0]                      
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 112, 112, 64) 9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 112, 112, 64) 256         conv2d_1[0][0]                   
____________________________________________________________________________________________

In [13]:
resnet50.compile(loss='mse',optimizer='adam')

In [22]:
# from keras.optimizers import Adam
# opt = keras.optimizers.Adam(0.7)
# resnet50.compile(optimizer=opt, loss="categorical_crossentropy", metrics=resnet50.compile(optimizer=opt, loss="categorical_crossentropy", metrics=['accuracy']))

NameError: name 'keras' is not defined

In [ ]:
history = resnet50.fit_generator(train_generator,steps_per_epoch=100,
                          validation_data=val_generator,validation_steps=15, 
                          epochs=50)

Epoch 1/50
  1/100 [..............................] - ETA: 1:09:46 - loss: 0.1974

# 여기서 부터 

---

### ResNet50 불러오기

In [1]:
from keras.applications import ResNet50
from keras.layers import Dense, Input, Activation
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
#from PIL import Image

Using TensorFlow backend.


In [2]:

'''train_datagen = ImageDataGenerator(featurewise_center=False,
                 samplewise_center=False,
                 featurewise_std_normalization=False,
                 samplewise_std_normalization=False,
                 zca_whitening=False,
                 rotation_range=5,
                 width_shift_range=0.05,
                 height_shift_range=0.05,
                 shear_range=0.2,
                 zoom_range=0.2,
                 channel_shift_range=0.,
                 fill_mode='nearest',
                 cval=0.,
                 horizontal_flip=True,
                 vertical_flip=False,
                 rescale=1/255,
                validation_split=0.33
                                  ) #rescale to [0-1], add zoom range of 0.2x and horizontal flip
'''
train_datagen = ImageDataGenerator(rescale=1/255,validation_split=0.33)
PATH="C:\FoodClassification\data"

train_generator = train_datagen.flow_from_directory(
        PATH,
        shuffle=True,
        seed=13,
        target_size=(224,224),
        batch_size=64,
        class_mode = 'categorical',
        subset="training")



val_generator = train_datagen.flow_from_directory(
        PATH,
        shuffle=True,
        seed=13,
        target_size=(224,224),
        batch_size=64,
        class_mode = 'categorical',
        subset="validation")

 

 

Found 13350 images belonging to 4 classes.
Found 6574 images belonging to 4 classes.


In [ ]:
input = Input(shape=(224, 224, 3))
model = ResNet50(input_tensor=input, include_top=True, weights=None, pooling='max')
model.summary()

In [3]:
input = Input(shape=(224, 224, 3))
model = ResNet50(input_tensor=input, include_top=False, weights=None, pooling='max')
model.trainable = False
model.summary()


Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
_________

In [4]:
# number of classes
K = 4
x = model.output
x = Dense(1024, name='fully', init='uniform')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dense(512, init='uniform')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
# Dense의 맨 앞부분 내가 필요한 category수만큼 적어주면 된다
x = Dense(K, activation='softmax', name='softmax')(x)
model = Model(model.input, x)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1024, name="fully", kernel_initializer="uniform")`
  after removing the cwd from sys.path.
C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(512, kernel_initializer="uniform")`
  import sys


In [5]:
model.compile(loss='mse',optimizer='adam',metrics=['accuracy'])
history = model.fit_generator(train_generator,steps_per_epoch=100,
                          validation_data=val_generator,validation_steps=15, 
                          epochs=50)


Epoch 1/50
 40/100 [===========>..................] - ETA: 28:34 - loss: 0.1938 - accuracy: 0.3344

C:\Users\admin\Anaconda3\lib\site-packages\PIL\Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


100/100 [==============================] - 3071s 31s/step - loss: 0.1782 - accuracy: 0.4077 - val_loss: 0.2678 - val_accuracy: 0.2729
Epoch 2/50
100/100 [==============================] - 3027s 30s/step - loss: 0.1406 - accuracy: 0.5742 - val_loss: 0.1925 - val_accuracy: 0.3375
Epoch 3/50
100/100 [==============================] - 3027s 30s/step - loss: 0.1266 - accuracy: 0.6297 - val_loss: 0.1725 - val_accuracy: 0.4771
Epoch 4/50
 99/100 [============================>.] - ETA: 29s - loss: 0.1257 - accuracy: 0.6351

C:\Users\admin\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 4. 
  warnings.warn(str(msg))


100/100 [==============================] - 3125s 31s/step - loss: 0.1257 - accuracy: 0.6353 - val_loss: 0.2216 - val_accuracy: 0.3115
Epoch 5/50
100/100 [==============================] - 3153s 32s/step - loss: 0.1088 - accuracy: 0.6869 - val_loss: 0.1702 - val_accuracy: 0.4094
Epoch 6/50
100/100 [==============================] - 3035s 30s/step - loss: 0.1101 - accuracy: 0.6878 - val_loss: 0.1997 - val_accuracy: 0.4187
Epoch 7/50
100/100 [==============================] - 3024s 30s/step - loss: 0.1212 - accuracy: 0.6439 - val_loss: 0.1787 - val_accuracy: 0.4130
Epoch 8/50
100/100 [==============================] - 3018s 30s/step - loss: 0.1000 - accuracy: 0.7148 - val_loss: 0.1769 - val_accuracy: 0.4448
Epoch 9/50
100/100 [==============================] - 3032s 30s/step - loss: 0.0967 - accuracy: 0.7212 - val_loss: 0.1243 - val_accuracy: 0.6031
Epoch 10/50
100/100 [==============================] - 3016s 30s/step - loss: 0.0831 - accuracy: 0.7648 - val_loss: 0.1752 - val_accuracy: 0.

ResourceExhaustedError: OOM when allocating tensor with shape[64,14,14,1024] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[{{node training/Adam/gradients/zeros_61}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


conda list

>conda install tensorflow-gpu